In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import Row
import numpy as np
import pandas as pd
import random
from datetime import datetime
import re

In [7]:
spark = SparkSession.builder \
    .master('local[*]') \
    .appName('IPDE') \
    .getOrCreate()
sc = spark.sparkContext

In [8]:
#population size (MUST BE multiple of workers)
NP=20
#Dimensions
D=3
#bounds
bound = 5
#workers = partitions = islands
workers = 4


In [91]:
# IMPO this function will go in another file
# now is here because is easier without the import
def islandsMap(index, iterator): 
    newPop = []
    oldPop = list(iterator)
    localItaration=0
    #stopping variable
    stop=False
    while(not stop):
        localItaration=localItaration+1
        #for each crhomosome in the population
        for i in oldPop:
            if(random.random()>0.5):
                newPop.append(i)
        if(localItaration>10):
            stop=True
    return iter(newPop)
    

In [92]:
#create population

#create the NP x D np.matrix with random numbers between +- bound
initialPop = np.random.random_sample((NP,D))*bound*2-bound


#create the NP x D dataframe with random numbers between +- bound
#df2 = pd.DataFrame(np.random.random_sample((NP,D))*bound*2-bound)

#partition the population
rdd = sc.parallelize(initialPop)
#create ann rdd with the partition number for each chromosome at random but evenly distributed
indexes = []
for i in range(0,workers):
    for k in range(0,int(NP/workers)):
        indexes.append(i)
random.shuffle(indexes)
# print("random indexes genereted: " , x)
index = sc.parallelize(indexes)
rdd = index.zip(rdd)
#partition by the key previously computed at random
rdd=rdd.partitionBy(workers)
print("number of partitions: ",rdd.getNumPartitions())
print("Partitions structure: {}".format(rdd.glom().collect()))
rdd.mapPartitionsWithIndex(islandsMap).collect()





number of partitions:  4
Partitions structure: [[(0, array([-1.68458273, -4.30414662,  3.91273121])), (0, array([ 1.80932347, -3.1435265 , -4.76100172])), (0, array([-2.00919164,  2.69779408,  4.30424398])), (0, array([ 0.11657619, -3.29743504,  1.24388455])), (0, array([-4.40972345,  2.33962134,  1.73187793]))], [(1, array([ 1.85575408,  1.23500413,  2.52187877])), (1, array([-2.37246052,  3.23531823,  3.71732519])), (1, array([ 4.23703437, -1.75655062,  3.72690506])), (1, array([ 4.48966961, -4.63855527,  3.07301684])), (1, array([-4.16297499,  1.32169145,  0.4600155 ]))], [(2, array([ 3.5985068 , -3.28622753, -4.61336879])), (2, array([ 4.53011251,  2.65240487,  3.84067035])), (2, array([ 1.94168573, -4.72611997,  0.3721346 ])), (2, array([ 1.45369639, -1.9617347 , -4.60448332])), (2, array([-1.21130658,  3.23412864, -1.49918941]))], [(3, array([ 0.62006722,  0.42262926, -1.87095673])), (3, array([ 0.27998049, -2.00541835,  4.84034378])), (3, array([ 2.32575096, -0.59669278,  4.5121

[(0, array([ 1.80932347, -3.1435265 , -4.76100172])),
 (0, array([-2.00919164,  2.69779408,  4.30424398])),
 (0, array([-4.40972345,  2.33962134,  1.73187793])),
 (0, array([ 1.80932347, -3.1435265 , -4.76100172])),
 (0, array([-2.00919164,  2.69779408,  4.30424398])),
 (0, array([ 0.11657619, -3.29743504,  1.24388455])),
 (0, array([-1.68458273, -4.30414662,  3.91273121])),
 (0, array([ 1.80932347, -3.1435265 , -4.76100172])),
 (0, array([-4.40972345,  2.33962134,  1.73187793])),
 (0, array([-1.68458273, -4.30414662,  3.91273121])),
 (0, array([-2.00919164,  2.69779408,  4.30424398])),
 (0, array([ 0.11657619, -3.29743504,  1.24388455])),
 (0, array([-4.40972345,  2.33962134,  1.73187793])),
 (0, array([-2.00919164,  2.69779408,  4.30424398])),
 (0, array([-4.40972345,  2.33962134,  1.73187793])),
 (0, array([-1.68458273, -4.30414662,  3.91273121])),
 (0, array([ 1.80932347, -3.1435265 , -4.76100172])),
 (0, array([-4.40972345,  2.33962134,  1.73187793])),
 (0, array([ 1.80932347, -3.